<a href="https://colab.research.google.com/github/matteogianferrari/homelens-ca/blob/main/model/notebooks/homelens_ca_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **HomeLens CA DL Model Research**

# **Setup**


---
**OS Environment Variables**

In [ ]:
from google.colab import userdata
import os

# Sets environ variables for GitHub
os.environ['GITHUB_TOKEN'] = userdata.get('GITHUB_TOKEN')
os.environ['USER'] = userdata.get('USER')

# Sets environ variables for MLFlow
os.environ['MLFLOW_TRACKING_USERNAME'] = userdata.get('USER')
os.environ['MLFLOW_TRACKING_PASSWORD'] = userdata.get('MLFLOW_TRACKING_PASSWORD')

---
**Libraries Install**

In [ ]:
!pip install dagshub mlflow pynvml --quiet

---
**Libraries imports**

In [ ]:
import tensorflow as tf

import mlflow
from mlflow.tracking import MlflowClient

import dagshub

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

---
**Dataset import**

In [ ]:
# Clones the repo and changes dir
!git clone https://${GITHUB_TOKEN}@github.com/${USER}/homelens-ca.git
%cd homelens-ca/

In [ ]:
# Changes dir
%cd model/data/dataset

# Imports the dataset and the labels
X = pd.read_csv('X.csv').to_numpy()
y = pd.read_csv('y.csv').to_numpy()

# Applies transformation to labels
y = np.log1p(y)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

---
**DagsHub and MLFlow**

In [ ]:
# Init DagsHub
dagshub.init(repo_owner='matteogianferrari', repo_name='homelens-ca', mlflow=True)
TRACKING_URI = "https://dagshub.com/matteogianferrari/homelens-ca.mlflow"

# Sets MLFlow tracking URI
mlflow.set_tracking_uri(TRACKING_URI)

# Sets MLFLow experiment name
experiment_name = "HomeLens CA-DL Research"

# **Research and Training DL Models**

---
**Model Hyperparameters**

Defines the Hyperparameters for the model.

In [ ]:
epochs = 400
initial_learning_rate = 1e-3
minimum_learning_rate = 1e-8
patience_es=50
patience_lr=15
decaying_factor=0.5
batch_size=128
validation_slip=0.1
loss_func='mse'

---
**Training and Testing**

In [ ]:
import pynvml

# Sets the MLFlow experiment
mlflow.set_experiment(experiment_name)

# Autolog
mlflow.keras.autolog()



# Defines the model
def build_dynamic_model(input_shape, n_repeats):
    """
    Builds a neural network model using TensorFlow's Functional API with repeated blocks.

    Args:
        input_shape (tuple): Shape of the input data.
        n_repeats (int): Number of times to repeat the block of layers.

    Returns:
        tf.keras.Model: A compiled Keras model.
    """
    # Input layer
    inputs = tf.keras.layers.Input(shape=input_shape)
    x = inputs

    # Define the repeated block
    for _ in range(n_repeats):
        x = tf.keras.layers.Dense(512, activation='relu', kernel_initializer='he_normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(0.2)(x)

        x = tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(0.2)(x)

        x = tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(0.2)(x)

    x = tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal')(x)
    x = tf.keras.layers.BatchNormalization()(x)

    # Output layer
    outputs = tf.keras.layers.Dense(1)(x)

    # Create the model
    model = tf.keras.Model(inputs, outputs)
    return model

# Example usage
n_repeats = 3
input_shape = (15,)
model = build_dynamic_model(input_shape, n_repeats)



# Set up a decaying learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)

# You can now compile your model using this optimizer
model.compile(
    optimizer=optimizer,
    loss=loss_func,
    metrics=['r2_score']
)

# Creates an Early-Stopping callback
es_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_r2_score',
    mode='max',
    patience=patience_es,
    restore_best_weights=True
)

lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=decaying_factor,
    patience=patience_lr,
    min_lr=minimum_learning_rate
)


# Train the model and log with MLFlow
with mlflow.start_run(log_system_metrics=True):
    mlflow.log_param("patience", patience_es)
    mlflow.log_param("loss_func", loss_func)

    history = model.fit(
        X_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=validation_slip,
        callbacks=[es_callback, lr_callback]
    )

    # Test the model and log with MLFlow
    test_loss, test_r2_score = model.evaluate(X_test, y_test)
    mlflow.log_metric("test_loss", test_loss)
    mlflow.log_metric("test_r2_score", test_r2_score)